 ## Importing libraries

In [1]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 7.2 MB/s eta 0:00:00


In [2]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.8/362.8 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 3.2 MB/s eta 0:00:00


In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error
from catboost import CatBoostRegressor
from sklearn.model_selection import GridSearchCV
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

# Create data

In [2]:
import pandas as pd
import numpy as np

# Set the date range for the time series data
date_range = pd.date_range(start="2013-01-01", periods=240, freq='D')
date_range

DatetimeIndex(['2013-01-01', '2013-01-02', '2013-01-03', '2013-01-04',
               '2013-01-05', '2013-01-06', '2013-01-07', '2013-01-08',
               '2013-01-09', '2013-01-10',
               ...
               '2013-08-19', '2013-08-20', '2013-08-21', '2013-08-22',
               '2013-08-23', '2013-08-24', '2013-08-25', '2013-08-26',
               '2013-08-27', '2013-08-28'],
              dtype='datetime64[ns]', length=240, freq='D')

In [3]:
# Set seed for reproducibility
np.random.seed(42)

# Create random data for 20 samples, each with a time series of length 240
# Separate data into 'input' and 'target' arrays

# Define number of samples and time series length
num_samples = 100
time_series_length = 240

# Generate input and target time series with random numbers
x_input = np.random.rand(num_samples, time_series_length)  # Input data (20 samples of length 240)
x_target = np.random.rand(num_samples, time_series_length)  # Target data (20 samples of length 240)

# Display a portion of the generated arrays as a sample
x_input[:5], x_target[:5]  # Showing first 5 samples as a representation of the data structure

(array([[0.37454012, 0.95071431, 0.73199394, ..., 0.02431597, 0.6454723 ,
         0.17711068],
        [0.94045858, 0.95392858, 0.91486439, ..., 0.55385428, 0.96930254,
         0.52309784],
        [0.62939864, 0.69574869, 0.45454106, ..., 0.96502691, 0.12429722,
         0.73086748],
        [0.93834046, 0.18123307, 0.06649627, ..., 0.04696597, 0.16628337,
         0.73803362],
        [0.08279867, 0.60315211, 0.24534911, ..., 0.96887786, 0.74965183,
         0.13008624]]),
 array([[0.68552802, 0.46757096, 0.0557069 , ..., 0.3197484 , 0.95160407,
         0.64564679],
        [0.45544964, 0.4683002 , 0.41074447, ..., 0.81409701, 0.50511242,
         0.39910194],
        [0.91545137, 0.23085355, 0.25697008, ..., 0.80880111, 0.54272237,
         0.00980053],
        [0.46984382, 0.29145165, 0.47755066, ..., 0.97480777, 0.60642649,
         0.72148541],
        [0.08800525, 0.62781243, 0.45384978, ..., 0.34686187, 0.80674474,
         0.79742693]]))

In [4]:
x_input.shape, x_target.shape

((100, 240), (100, 240))

In [5]:
X_train = x_input[:60]
y_train = x_target[:60]
X_val = x_input[60:80]
y_val = x_target[60:80]
X_test = x_input[80:]
y_test = x_target[80:]

In [6]:
X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape, y_test.shape

((60, 240), (60, 240), (20, 240), (20, 240), (20, 240), (20, 240))

# Catboost

In [7]:
import numpy as np
import optuna
import catboost as cb
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

In [ ]:
def objective(trial):
    # Define CatBoost parameters for multi-target regression
    param = {
        "objective": "MultiRMSE",  # Multi-target regression objective
        "depth": trial.suggest_int("depth", 1, 12),
        "boosting_type": "Plain",
        "bootstrap_type": trial.suggest_categorical(
            "bootstrap_type", ["Bayesian", "Bernoulli", "MVS"]
        ),
        "used_ram_limit": "14gb",
        "task_type": "GPU"  # Use GPU for training
    }

    if param["bootstrap_type"] == "Bayesian":
        param["bagging_temperature"] = trial.suggest_float("bagging_temperature", 0, 10)
    elif param["bootstrap_type"] == "Bernoulli":
        param["subsample"] = trial.suggest_float("subsample", 0.1, 1)

    # Initialize CatBoost Regressor for multi-target regression
    gbm = cb.CatBoostRegressor(**param, verbose=0)

    # Fit on the training data as 2D inputs with multi-dimensional targets
    gbm.fit(X_train, y_train, eval_set=[(X_val, y_val)], early_stopping_rounds=100)

    # Predictions and mean squared error calculation across all targets
    preds = gbm.predict(X_val)
    mse = mean_squared_error(y_val, preds, multioutput='raw_values').mean()  # Average MSE over all time steps

    return mse

if __name__ == "__main__":
    study = optuna.create_study(direction="minimize")  # minimize MSE
    study.optimize(objective, n_trials=100, timeout=600)

    print("Number of finished trials: {}".format(len(study.trials)))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: {}".format(trial.value))

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

[I 2024-11-10 04:47:17,823] A new study created in memory with name: no-name-dc187344-a139-469d-8d04-259b8501155c
